In [ ]:
import pandas as pd
import time
from openai import OpenAI

client = OpenAI(
    api_key="YOUR KEY",  
    base_url="https://api.nuwaapi.com/v1"  # A proxy link for the GPT API that allows you to use the GPT API directly.
)

df = pd.read_csv("./Relevant dataset/adverse_events_count_ge_5.csv")
ae_list = df["Adverse Event"].astype(str).tolist()


fixed_categories = [
    "Blood and lymphatic system disorders",
    "Cardiac disorders",
    "Congenital, familial and genetic disorders",
    "Ear and labyrinth disorders",
    "Endocrine disorders",
    "Eye disorders",
    "Gastrointestinal disorders",
    "General disorders and administration site conditions",
    "Hepatobiliary disorders",
    "Immune system disorders",
    "Infections and infestations",
    "Injury, poisoning and procedural complications",
    "Investigations",
    "Metabolism and nutrition disorders",
    "Musculoskeletal and connective tissue disorders",
    "Neoplasms benign, malignant and unspecified (incl cysts and polyps)",
    "Nervous system disorders",
    "Pregnancy, puerperium and perinatal conditions",
    "Product issues",
    "Psychiatric disorders",
    "Renal and urinary disorders",
    "Reproductive system and breast disorders",
    "Respiratory, thoracic and mediastinal disorders",
    "Skin and subcutaneous tissue disorders",
    "Social circumstances",
    "Surgical and medical procedures",
    "Vascular disorders",
]

gpt_results = []

for ae in ae_list:
    prompt = f"""
You are a medical classification expert. Your task is to assign the following adverse event to one of the 27 fixed categories listed below, which come from MedDRA SOC.

You need refer to the Definition of the definition of those categories when you give me the category of adverse event. 

You must choose ONLY ONE category. Do not explain or list multiple options. Only return the category name exactly as written.

Categories:
{chr(10).join(fixed_categories)}

Adverse Event: {ae}

Respond ONLY with the most appropriate category name from the list above.
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a medical classification expert."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            max_tokens=20
        )

        category = response.choices[0].message.content.strip()
        gpt_results.append(category)
        print(f"{ae} → {category}")

    except Exception as e:
        gpt_results.append("Uncategorized")
        print(f"Error on {ae}: {str(e)}")

    time.sleep(1.2)

df["GPT_Category"] = gpt_results
df.to_csv("./Relevant dataset/adverse_event_classified_by_gpt.csv", index=False)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import cohen_kappa_score


df1 = pd.read_csv('./Relevant dataset/ae_to_soc_cluster.csv', dtype=str)
df2 = pd.read_csv('./Relevant dataset/adverse_event_classified_by_gpt.csv', dtype=str)

merged = pd.merge(
    df1,
    df2,
    left_on='Adverse Events',
    right_on='Adverse Event',
    how='inner'
)

le = LabelEncoder()
le.fit(pd.concat([merged['SOC Cluster'], merged['GPT_Category']]))

y_true = le.transform(merged['SOC Cluster'])
y_pred = le.transform(merged['GPT_Category'])

kappa = cohen_kappa_score(y_true, y_pred)
print(f"Cohen's Kappa: {kappa:.4f}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df1 = pd.read_csv('./Relevant dataset/ae_to_soc_cluster.csv', dtype=str)
df2 = pd.read_csv('./Relevant dataset/adverse_event_classified_by_gpt.csv', dtype=str)

merged = pd.merge(
    df1,
    df2,
    left_on='Adverse Events',
    right_on='Adverse Event',
    how='inner'
)

merged['error'] = merged['SOC Cluster'] != merged['GPT_Category']

total = len(merged)
correct = (~merged['error']).sum()
overall_accuracy = correct / total * 100
print(f"Overall accuracy: {overall_accuracy:.2f}%")

summary = (
    merged
    .groupby('SOC Cluster')['error']
    .agg(total='size', errors='sum')
)
summary['correct'] = summary['total'] - summary['errors']
summary['accuracy'] = summary['correct'] / summary['total'] * 100
summary = summary.sort_values('accuracy', ascending=False)

print("\nPer-cluster summary:")
print(summary[['total', 'errors', 'correct', 'accuracy']])

plt.figure(figsize=(8, 10))
sns.barplot(
    x='accuracy',
    y=summary.index,
    data=summary,
    palette='Greens_r' 
)


for idx, (cluster, row) in enumerate(summary.iterrows()):
    plt.text(
        row.accuracy + 1,   
        idx,
        f"{row.accuracy:.1f}%",
        va='center'
    )

plt.xlabel('Accuracy (%)', fontsize=12)
plt.ylabel('SOC Cluster', fontsize=12)
plt.title('Accuracy by SOC Cluster', fontsize=14, fontweight='bold')
plt.xlim(0, 100)
plt.tight_layout()
plt.show()
